In [90]:
import pandas as pd
from sodapy import Socrata
from geopy import distance
import geopandas
import matplotlib.pyplot as plt
import folium

In [35]:
pip install geopandas --user

     |████████████████████████████████| 931kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 27.2MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 16.4MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [97]:
def writeLatLong():
    try:
        for data in results:
            data['lat'] = 0
            data['lon'] = 0
            data.setdefault('location', 'not found')
            if 'latitude' in data['location']:
                lat = float(data['location']['latitude'])
                lon = float(data['location']['longitude'])
                data['lat'] = lat
                data['lon'] = lon
            elif 'latitude' not in data['location']:
                data['coords'] = 'not found'
    except KeyError:
        print('key error')

onondaga_county = (43.0268, 76.1784)
map = folium.Map(location=(43.0268, -76.1784), zoom_start=7)
        
client = Socrata('data.ny.gov',
                  'LZ8n22WcfBkEDojOKGKIG9tsm',
                  username="rmckee@syr.edu",
                  password="Syr@1923")

client2 = Socrata('data.ny.gov',
                  'LZ8n22WcfBkEDojOKGKIG9tsm',
                  username="rmckee@syr.edu",
                  password="Syr@1923")

covid_results = client2.get('xdss-u53e')

covid_results_df = pd.DataFrame.from_records(covid_results)

results = client.get("9a8c-vfzj", county='Onondaga')

writeLatLong()

results_df = pd.DataFrame.from_records(results)

for row in results_df.to_records():
    pos = (row['lat'], row['lon'])
    message = row['entity_name']
    marker = folium.Marker(location=pos, 
                    popup=message
                          )
    map.add_child(marker)

map

In [92]:
results_df

,county,license_number,operation_type,estab_type,entity_name,dba_name,street_number,street_name,address_line_2,address_line_3,city,state,zip_code,square_footage,location,lat,lon,coords
0,Onondaga,732921,Store,A,HICKORY FARMS INC,HICKORY FARMS #11397,9090,DESTINY USA DR,,,SYRACUSE,NY,13204,0,"{'latitude': '43.067697', 'longitude': '-76.17...",43.067697,-76.170515,NaN
1,Onondaga,310706,Store,JAC,MURSHED ABDO,DREAM FOOD MARKET,518,WOLF ST,,,SYRACUSE,NY,13208,1500,"{'latitude': '43.074355', 'longitude': '-76.16...",43.074355,-76.161879,NaN
2,Onondaga,705210,Store,JAC,PETR-ALL PETROLEUM CORP,EXPRESS MART 328,2723,BREWERTON RD,,,MATTYDALE,NY,13211,2100,"{'latitude': '43.105369', 'longitude': '-76.14...",43.105369,-76.146029,NaN
3,Onondaga,720768,Store,JAC,SPEEDWAY LLC,SPEEDWAY #07899,407,E GENESEE ST,,,FAYETTEVILLE,NY,13066,1500,"{'latitude': '43.029623', 'longitude': '-76.00...",43.029623,-76.006511,NaN
4,Onondaga,310545,Store,JAC,MESSINEO ENTERPRISES INC,KEENS CONVIENCE,7245,STATE FAIR BLVD,,,SYRACUSE,NY,13209,2000,"{'latitude': '43.120684', 'longitude': '-76.28...",43.120684,-76.285883,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,Onondaga,310627,Store,A,REAL DEALS OF AUBURN INC,REAL DEALS,2265,DOWNER ST RIVER MALL,,,BALDWINSVILLE,NY,13027,10000,"{'human_address': '{""address"": ""2265 DOWNER ST...",0.000000,0.000000,not found
641,Onondaga,311015,Store,A,ALDI INC,ALDI #47,6633,MANLIUS CENTER RD,,,EAST SYRACUSE,NY,13057,5000,"{'latitude': '43.063388', 'longitude': '-76.05...",43.063388,-76.057719,NaN
642,Onondaga,310407,Store,JABC,PRICE CHOPPER OPERATING CO INC,PRICE CHOPPER 172,2515,ERIE BLVD E,,,SYRACUSE,NY,13224,100000,"{'latitude': '43.054021', 'longitude': '-76.10...",43.054021,-76.102319,NaN
643,Onondaga,310654,Store,JABC,PRICE CHOPPER OPERATING CO INC,PRICE CHOPPER 199,5701,E CIRCLE DR,,,CICERO,NY,13039,68000,"{'latitude': '43.148251', 'longitude': '-76.12...",43.148251,-76.121642,NaN


In [88]:
gdf = geopandas.GeoDataFrame(
    results_df, geometry=geopandas.points_from_xy(results_df.lat, results_df.lon))